In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSR_avg%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFDSR_avg%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)
tempF = InterfaceData(size=inodes, value=300.0)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = RelativeCovergenceMonitor(tol=1e-5)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 8000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FDSR_avg%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FDSR_avg%i.log' % k, mode='a')

abtemp_prev = abTempF.curr.copy()
temps_prev = TempS.curr.copy()

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    abtemp_prev[:] = abTempF.curr
    temps_prev[:] = TempS.curr
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve the wall temperature for convergence testing
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = 0.5*(abTempF.curr[f2s]+abtemp_prev[f2s])
        # update solid flux interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        TempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = 0.5*(TempS.curr[s2f] + temps_prev[s2f])
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF)
                under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=5.
step=1, pc_iterations=4.
step=2, pc_iterations=4.
step=3, pc_iterations=4.
step=4, pc_iterations=4.
step=5, pc_iterations=4.
step=6, pc_iterations=4.
step=7, pc_iterations=4.
step=8, pc_iterations=4.
step=9, pc_iterations=4.
step=10, pc_iterations=4.
step=11, pc_iterations=4.
step=12, pc_iterations=3.
step=13, pc_iterations=3.
step=14, pc_iterations=4.
step=15, pc_iterations=3.
step=16, pc_iterations=3.
step=17, pc_iterations=3.
step=18, pc_iterations=3.
step=19, pc_iterations=3.
step=20, pc_iterations=3.
step=21, pc_iterations=3.
step=22, pc_iterations=3.
step=23, pc_iterations=3.
step=24, pc_iterations=3.
step=25, pc_iterations=3.
step=26, pc_iterations=3.
step=27, pc_iterations=3.
step=28, pc_iterations=3.
step=29, pc_iterations=3.
step=30, pc_iterations=3.
step=31, pc_iterations=3.
step=32, pc_iterations=3.
step=33, pc_iterations=3.
step=34, pc_iterations=3.
step=35, pc_iterations=3.
step=36, pc_iterations=3.
step=37, pc_iterations=3.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFDSR_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.9371447  0.93848452 0.9406245  0.94269771 0.94465534 0.9464125
 0.94805467 0.94957168 0.95099095 0.95236319 0.95364682 0.954873
 0.95603305 0.95714983 0.95820873 0.95922788 0.96019886 0.96113688
 0.96203678 0.96289838 0.96372923 0.96452642 0.96529744 0.96603776
 0.96674854 0.96743321 0.96809595 0.9687341  0.96935111 0.96994561
 0.97051795 0.97107089 0.97160511 0.97211862 0.97261807 0.97309839
 0.97356092 0.97400724 0.97443998 0.97485714 0.97526061 0.9756501
 0.97602538 0.97638758 0.97673713 0.97707456 0.97739967 0.97771397
 0.97801612 0.97830749 0.9785873  0.97885771 0.97911753 0.97936637
 0.97960618 0.97983524 0.98005559 0.98026566 0.98046628 0.98065811
 0.98084013 0.98101314 0.98117646 0.98133143 0.98147672 0.98161234
 0.98173967 0.98185659 0.98196473 0.98206306 0.98215162 0.98223059
 0.98229876 0.9823572  0.98240461 0.98244167 0.98246668 0.98248036
 0.98248248 0.98247119 0.98244766 0.98240958 0.98235725 0.98229067
 0.98220731 0.98210731 0.98198839 0.98185077 0.98169194 0.98150914